In [1]:
from my_ai_utils import *
import keras
import tensorflow as tf

In [2]:
# Example input data
batch_size = 20
input_size = 4
output_size = 3

X = np.random.randn(batch_size, input_size).astype(np.float32)
y = np.random.randn(batch_size, output_size).astype(np.float32)

In [3]:
# Initialize the models
model = tf.keras.layers.Dense(output_size, activation="sigmoid")
model.build(input_shape=(None, input_size))
weights = model.get_weights()
print(weights)
custom_dense = Dense(in_features=input_size, out_features=output_size, load_weights=weights, activation="sigmoid")

[array([[-0.19376117,  0.5165348 ,  0.12137866],
       [ 0.5977504 , -0.16174674,  0.02624691],
       [ 0.5346416 , -0.4416696 , -0.11375374],
       [-0.6169945 ,  0.7592577 ,  0.28805995]], dtype=float32), array([0., 0., 0.], dtype=float32)]


In [4]:
# Forward pass test
y_pred = model(X)
custom_y_pred = custom_dense(X)
print("Forward pass output:", y_pred.shape, y_pred[0].numpy())  # Expected shape: (batch_size, output_size)
print("Forward custom pass output :", custom_y_pred.shape, custom_y_pred[0])

Forward pass output: (20, 3) [0.21098357 0.728575   0.5840468 ]
Forward custom pass output : (20, 3) [0.21098359 0.72857493 0.5840468 ]


In [5]:
# Define a simple loss function
loss_fn = tf.keras.losses.MeanSquaredError()

# Compute the loss
with tf.GradientTape() as tape:
    y_pred = model(X)
    loss = loss_fn(y, y_pred)

print("Loss:", loss.numpy())

### custom
custom_loss, custom_gradient = Loss()(custom_y_pred, y)
print("Custom loss:", custom_loss)

Loss: 1.2588962
Custom loss: 1.2588961


In [6]:
# Backward pass test (compute gradients)
_, custom_params_updates = custom_dense.backward(custom_gradient)
custom_params_updates = Adam(lr=0.01)(custom_params_updates, 0, step=0, epoch=0)
custom_dense.update_params(custom_params_updates)

gradients = tape.gradient(loss, model.trainable_variables)
print("Gradients brute :", gradients)
print("custom Gradients brute :", custom_params_updates)

# Optionally, perform a gradient update to check if loss decreases
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)

# Perform a single optimization step
optimizer.apply_gradients(zip(gradients, model.trainable_variables))

Gradients brute : [<tf.Tensor: shape=(4, 3), dtype=float32, numpy=
array([[ 0.05768738,  0.0405526 , -0.00604385],
       [-0.07270585,  0.0677229 , -0.02270087],
       [ 0.03265507, -0.00989298,  0.03027873],
       [-0.04756082,  0.04349323,  0.06124633]], dtype=float32)>, <tf.Tensor: shape=(3,), dtype=float32, numpy=array([0.04556761, 0.02427512, 0.05740745], dtype=float32)>]
custom Gradients brute : [array([[ 0.01,  0.01, -0.01],
       [-0.01,  0.01, -0.01],
       [ 0.01, -0.01,  0.01],
       [-0.01,  0.01,  0.01]], dtype=float32), array([0.01, 0.01, 0.01], dtype=float32)]


<tf.Variable 'UnreadVariable' shape=() dtype=int64, numpy=1>

In [7]:
# Compute the loss again to check if it has decreased
with tf.GradientTape() as tape:
    y_pred = model(X)
    new_loss = loss_fn(y, y_pred)

custom_y_pred = custom_dense(X)
new_custom_loss, _ = Loss()(custom_y_pred, y)

print("New Loss:", new_loss.numpy())
print("New custom Loss:", new_custom_loss)

assert new_loss < loss, "Loss did not decrease after gradient update"
assert new_custom_loss < custom_loss, "Custom Loss did not decrease after gradient update"

New Loss: 1.2527087
New custom Loss: 1.2527082
